In [1]:
import sys
sys.path.append('..')
from src.utilities import *
from src.models.train_model import train_model
from src.models.param_opt import bayes_parameter_opt_lgb

## 3. Modelling

### 3.1 Load master table

In [2]:
master = pd.read_csv(os.path.join(processed_path, 'master.csv'))

### 3.2 Train / val / test split

In [3]:
x_train = master[(master.sales.isna()==False)].drop(columns = ['region', 'brand']).copy()
x_train = x_train[x_train.month >= '2020-06']
x_train['train'] =  (x_train.month <= '2021-06').astype(float)
x_train.drop(columns = 'month', inplace = True)

### 3.3 Bayesian parameter search

In [4]:
# best_params = bayes_parameter_opt_lgb(X = x_train.drop(columns = ['sales', 'train']), y = x_train.sales,
#                                       init_round=15, opt_round=10, n_folds=5, random_seed=6, n_estimators=10000, 
#                                       learning_rate=0.01, save_path = '', metric = 'mae').max['params']
# best_params

### 3.4 Model training

In [4]:
##### Define best parameters found earlier
best_params = {'metric': 'mae',
               'bagging_fraction': 0.8428677772772322,
               'feature_fraction': 0.8315752549749759,
               'lambda_l1': 0.9445413063248725,
               'lambda_l2': 2.736382499255929,
               'max_depth': 17,
               'min_child_weight': 7.0731275492417005,
               'min_split_gain': 0.0062674733053891315,
               'num_leaves': 42}

##### Train a quantile regression lgb at different alpha levels
quantile_alphas = [0.1, 0.5, 0.9]
lgb_quantiles = {}

for alpha in quantile_alphas:
    current_model = train_model(x_train, target_name = 'sales', 
                                model_type = 'lgb-quantile', 
                                quantile_alpha = alpha,
                                params = best_params, 
                                metric = 'rmse', 
                                split = 'in_sample',
                                save_path = os.path.join(models_path, 
                                                         'model2_quantile_'+ str(alpha)+'.pkl'))
    lgb_quantiles[alpha] = current_model
    
##### Visualize feature contributions
feature_contributions = pd.DataFrame({'feature': x_train.drop(columns = ['sales', 'train']).columns, 
                                      'gain': lgb_quantiles[0.5].feature_importance(importance_type = 'gain'),
                                      'split': lgb_quantiles[0.5].feature_importance(importance_type = 'split')
                                     }).sort_values('gain', ascending = False)

feature_contributions[feature_contributions.gain>0][:40]

# 0.2 - 1876.25  1789.02
# 0.5 - 1317.57  1372.7
# 0.8 -  923.42  1110.04

# 0.2 - 723.847  868.854
# 0.5 - 344.03  617.231
# 0.8 -  365.334  693.425

####################     training with  4077      ####################
####################     validating with  453      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026823 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32922
[LightGBM] [Info] Number of data points in the train set: 4077, number of used features: 479
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l1: 1790.71	valid_1's l1: 1727.51
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's l1: 1771.78	valid_1's l1: 1708.78
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training's l1: 1754.49	valid_1's l1: 1691.43
[4]	training's l1: 1739.93	valid_1's l1: 1676.42
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	tra

[152]	training's l1: 1168.59	valid_1's l1: 1166.21
[153]	training's l1: 1168.34	valid_1's l1: 1166.03
[154]	training's l1: 1167.22	valid_1's l1: 1165.08
[155]	training's l1: 1167.2	valid_1's l1: 1165.15
[156]	training's l1: 1166.7	valid_1's l1: 1164.79
[157]	training's l1: 1164.62	valid_1's l1: 1163.05
[158]	training's l1: 1164.11	valid_1's l1: 1162.75
[159]	training's l1: 1163.33	valid_1's l1: 1162.31
[160]	training's l1: 1162.89	valid_1's l1: 1161.95
[161]	training's l1: 1160.74	valid_1's l1: 1160.05
[162]	training's l1: 1159.81	valid_1's l1: 1159.15
[163]	training's l1: 1156.92	valid_1's l1: 1158.07
[164]	training's l1: 1156.45	valid_1's l1: 1157.62
[165]	training's l1: 1156.23	valid_1's l1: 1157.59
[166]	training's l1: 1155.36	valid_1's l1: 1157.01
[167]	training's l1: 1154.41	valid_1's l1: 1156.25
[168]	training's l1: 1153.86	valid_1's l1: 1155.83
[169]	training's l1: 1152.98	valid_1's l1: 1155.16
[170]	training's l1: 1152.88	valid_1's l1: 1155.13
[171]	training's l1: 1152.36	vali

[313]	training's l1: 1108.88	valid_1's l1: 1122.42
[314]	training's l1: 1108.85	valid_1's l1: 1122.4
[315]	training's l1: 1108.78	valid_1's l1: 1122.42
[316]	training's l1: 1108.73	valid_1's l1: 1122.4
[317]	training's l1: 1108.66	valid_1's l1: 1122.38
[318]	training's l1: 1108.6	valid_1's l1: 1122.37
[319]	training's l1: 1108.56	valid_1's l1: 1122.35
[320]	training's l1: 1108.42	valid_1's l1: 1122.24
[321]	training's l1: 1108.39	valid_1's l1: 1122.22
[322]	training's l1: 1108.34	valid_1's l1: 1122.22
[323]	training's l1: 1108.25	valid_1's l1: 1122.17
[324]	training's l1: 1108.22	valid_1's l1: 1122.18
[325]	training's l1: 1108.2	valid_1's l1: 1122.16
[326]	training's l1: 1108.1	valid_1's l1: 1122.11
[327]	training's l1: 1108.06	valid_1's l1: 1122.05
[328]	training's l1: 1107.65	valid_1's l1: 1121.7
[329]	training's l1: 1107.31	valid_1's l1: 1121.54
[330]	training's l1: 1107.27	valid_1's l1: 1121.53
[331]	training's l1: 1107.23	valid_1's l1: 1121.51
[332]	training's l1: 1107.17	valid_1'

[481]	training's l1: 1089.67	valid_1's l1: 1110.25
[482]	training's l1: 1089.42	valid_1's l1: 1110.05
[483]	training's l1: 1089.26	valid_1's l1: 1109.92
[484]	training's l1: 1088.9	valid_1's l1: 1109.71
[485]	training's l1: 1088.86	valid_1's l1: 1109.68
[486]	training's l1: 1088.78	valid_1's l1: 1109.61
[487]	training's l1: 1088.71	valid_1's l1: 1109.51
[488]	training's l1: 1088.68	valid_1's l1: 1109.49
[489]	training's l1: 1088.6	valid_1's l1: 1109.46
[490]	training's l1: 1088.56	valid_1's l1: 1109.45
[491]	training's l1: 1088.54	valid_1's l1: 1109.44
[492]	training's l1: 1088.38	valid_1's l1: 1109.2
[493]	training's l1: 1088.36	valid_1's l1: 1109.2
[494]	training's l1: 1088.35	valid_1's l1: 1109.19
[495]	training's l1: 1088.3	valid_1's l1: 1109.16
[496]	training's l1: 1088.28	valid_1's l1: 1109.15
[497]	training's l1: 1088.27	valid_1's l1: 1109.14
[498]	training's l1: 1088.24	valid_1's l1: 1109.14
[499]	training's l1: 1088.08	valid_1's l1: 1109
[500]	training's l1: 1087.84	valid_1's 

[651]	training's l1: 1074.99	valid_1's l1: 1099.4
[652]	training's l1: 1074.98	valid_1's l1: 1099.39
[653]	training's l1: 1074.96	valid_1's l1: 1099.38
[654]	training's l1: 1074.93	valid_1's l1: 1099.38
[655]	training's l1: 1074.9	valid_1's l1: 1099.36
[656]	training's l1: 1074.88	valid_1's l1: 1099.35
[657]	training's l1: 1074.86	valid_1's l1: 1099.36
[658]	training's l1: 1074.82	valid_1's l1: 1099.32
[659]	training's l1: 1074.8	valid_1's l1: 1099.32
[660]	training's l1: 1074.68	valid_1's l1: 1099.24
[661]	training's l1: 1074.27	valid_1's l1: 1099
[662]	training's l1: 1074.25	valid_1's l1: 1098.98
[663]	training's l1: 1074.1	valid_1's l1: 1098.86
[664]	training's l1: 1074.08	valid_1's l1: 1098.85
[665]	training's l1: 1073.86	valid_1's l1: 1098.89
[666]	training's l1: 1073.57	valid_1's l1: 1098.7
[667]	training's l1: 1073.48	valid_1's l1: 1098.75
[668]	training's l1: 1073.42	valid_1's l1: 1098.64
[669]	training's l1: 1073.18	valid_1's l1: 1098.43
[670]	training's l1: 1073.11	valid_1's 

...

[384]	training's l1: 717.664	valid_1's l1: 959.598
[385]	training's l1: 717.558	valid_1's l1: 959.537
[386]	training's l1: 717.457	valid_1's l1: 959.439
[387]	training's l1: 717.407	valid_1's l1: 959.408
[388]	training's l1: 717.193	valid_1's l1: 959.412
[389]	training's l1: 717.121	valid_1's l1: 959.366
[390]	training's l1: 717.205	valid_1's l1: 959.458
[391]	training's l1: 717.443	valid_1's l1: 959.373
[392]	training's l1: 717.208	valid_1's l1: 959.116
[393]	training's l1: 716.991	valid_1's l1: 958.955
[394]	training's l1: 716.744	valid_1's l1: 958.773
[395]	training's l1: 716.632	valid_1's l1: 958.711
[396]	training's l1: 716.556	valid_1's l1: 958.668
[397]	training's l1: 716.399	valid_1's l1: 958.544
[398]	training's l1: 716.326	valid_1's l1: 958.573
[399]	training's l1: 715.995	valid_1's l1: 958.381
[400]	training's l1: 715.875	valid_1's l1: 958.345
[401]	training's l1: 715.749	valid_1's l1: 958.232
[402]	training's l1: 715.644	valid_1's l1: 958.15
[403]	training's l1: 715.332	val

[547]	training's l1: 701.859	valid_1's l1: 948.635
[548]	training's l1: 701.777	valid_1's l1: 948.577
[549]	training's l1: 701.727	valid_1's l1: 948.53
[550]	training's l1: 701.553	valid_1's l1: 948.403
[551]	training's l1: 701.467	valid_1's l1: 948.373
[552]	training's l1: 701.313	valid_1's l1: 948.244
[553]	training's l1: 701.243	valid_1's l1: 948.211
[554]	training's l1: 701.209	valid_1's l1: 948.195
[555]	training's l1: 701.297	valid_1's l1: 948.322
[556]	training's l1: 701.216	valid_1's l1: 948.256
[557]	training's l1: 701.132	valid_1's l1: 948.184
[558]	training's l1: 701.019	valid_1's l1: 948.1
[559]	training's l1: 700.955	valid_1's l1: 948.091
[560]	training's l1: 700.906	valid_1's l1: 948.059
[561]	training's l1: 700.853	valid_1's l1: 948.014
[562]	training's l1: 700.742	valid_1's l1: 947.939
[563]	training's l1: 700.778	valid_1's l1: 947.998
[564]	training's l1: 700.733	valid_1's l1: 947.978
[565]	training's l1: 700.666	valid_1's l1: 947.924
[566]	training's l1: 700.603	valid

[714]	training's l1: 691.362	valid_1's l1: 940.98
[715]	training's l1: 691.328	valid_1's l1: 940.964
[716]	training's l1: 691.213	valid_1's l1: 940.864
[717]	training's l1: 691.055	valid_1's l1: 940.714
[718]	training's l1: 691.049	valid_1's l1: 940.708
[719]	training's l1: 690.985	valid_1's l1: 940.651
[720]	training's l1: 690.976	valid_1's l1: 940.635
[721]	training's l1: 690.914	valid_1's l1: 940.594
[722]	training's l1: 690.893	valid_1's l1: 940.577
[723]	training's l1: 690.658	valid_1's l1: 940.36
[724]	training's l1: 690.629	valid_1's l1: 940.34
[725]	training's l1: 690.637	valid_1's l1: 940.293
[726]	training's l1: 690.544	valid_1's l1: 940.25
[727]	training's l1: 690.516	valid_1's l1: 940.255
[728]	training's l1: 690.479	valid_1's l1: 940.237
[729]	training's l1: 690.225	valid_1's l1: 939.904
[730]	training's l1: 690.209	valid_1's l1: 939.89
[731]	training's l1: 690.091	valid_1's l1: 939.776
[732]	training's l1: 690.026	valid_1's l1: 939.74
[733]	training's l1: 689.967	valid_1'

[886]	training's l1: 682.717	valid_1's l1: 935.157
[887]	training's l1: 682.735	valid_1's l1: 935.185
[888]	training's l1: 682.705	valid_1's l1: 935.162
[889]	training's l1: 682.673	valid_1's l1: 935.144
[890]	training's l1: 682.639	valid_1's l1: 935.109
[891]	training's l1: 682.584	valid_1's l1: 935.074
[892]	training's l1: 682.53	valid_1's l1: 935.048
[893]	training's l1: 682.515	valid_1's l1: 935.041
[894]	training's l1: 682.491	valid_1's l1: 935.024
[895]	training's l1: 682.464	valid_1's l1: 935.005
[896]	training's l1: 682.438	valid_1's l1: 934.988
[897]	training's l1: 682.424	valid_1's l1: 934.985
[898]	training's l1: 682.385	valid_1's l1: 934.971
[899]	training's l1: 682.338	valid_1's l1: 934.957
[900]	training's l1: 682.319	valid_1's l1: 934.944
[901]	training's l1: 682.32	valid_1's l1: 934.937
[902]	training's l1: 682.301	valid_1's l1: 934.925
[903]	training's l1: 682.283	valid_1's l1: 934.911
[904]	training's l1: 682.264	valid_1's l1: 934.9
[905]	training's l1: 682.219	valid_

,feature,gain,split
147,sales_region_b3market_trend_6mo,7001.519396,175
0,month_indicator,2334.828050,49
161,sales_region_b12market_trend_6mo,1272.261502,85
8,sales_univ_brand_1mo,1148.820944,89
123,sales_univ_brand_cumstd,1039.694046,106
136,sales_region_b3_cummin,394.263071,50
142,sales_region_b3market_6mo,377.345817,39
191,region_nhcp_intmed,375.147360,136
182,region_n_intmedandgen_perperson,347.067349,108
128,sales_region_b3_6mo,343.670127,54


### 3.5 Prediction storage

In [5]:
submission = master[(master.sales.isna()) & (master.month >= '2020-07')].copy()
submission['sales'] = lgb_quantiles[0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['lower'] = lgb_quantiles[0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['upper'] = lgb_quantiles[0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales', 'lower']))
submission = submission[['month', 'region', 'brand', 'sales', 'lower', 'upper']]

submission.loc[submission.sales < 0, 'sales'] = 0
submission.loc[submission.lower < 0, 'lower'] = 0
submission.loc[submission.upper < 0, 'upper'] = 0
submission.head()

,month,region,brand,sales,lower,upper
2714,2020-07,region_151,brand_1,22.124358,0.0,116.417853
2715,2020-07,region_151,brand_2,22.124358,0.0,116.417853
2716,2020-07,region_152,brand_1,64.516461,0.0,206.143892
2717,2020-07,region_152,brand_2,64.516461,0.0,206.143892
2718,2020-07,region_153,brand_1,24.644612,0.0,842.533485


In [6]:
submission.to_csv(os.path.join(results_path, 'submission10_team46.csv'), index = False)

In [7]:
(submission['upper'] - submission['lower']).mean()

# Submission 2 difference - 1815.437323083508
# Submission 3 difference - 1613.6602449420075
# Submission 4 difference - 1145.0976140382802
# Submission 5 difference - 1132.1785576554485
#1222.15

1927.039236379451

In [8]:
preds_train = pd.DataFrame({'month': master[(master.sales.isna()==False) & (master.month >= '2020-07')].month,
                            'region': master[(master.sales.isna()==False) & (master.month >= '2020-07')].region,
                            'brand': master[(master.sales.isna()==False) & (master.month >= '2020-07')].brand,
                            'predictions': lgb_quantiles[0.5].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'lower': lgb_quantiles[0.1].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'upper': lgb_quantiles[0.9].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'sales': master[(master.sales.isna()==False) & (master.month >= '2020-07')].sales
                           })

preds_train.loc[preds_train.predictions < 0, 'predictions'] = 0
preds_train.loc[preds_train.lower < 0, 'lower'] = 0
preds_train.loc[preds_train.upper < 0, 'upper'] = 0

width = abs(preds_train['upper'] - preds_train['lower'])
below = (preds_train['lower'] - preds_train['sales'])*(preds_train['lower'] > preds_train['sales'])
above = (preds_train['sales'] - preds_train['upper'])*(preds_train['upper'] < preds_train['sales'])
outside = (2/0.2)*(below + above)

In [9]:
preds_train['outside'] = (preds_train.predictions < preds_train.lower)  | (preds_train.predictions > preds_train.upper)
preds_train.outside.mean()

0.11660359508041628

In [9]:
width.mean()
# 0.25 988.73
# 0.2  1090.40
# 0.1 1827.48

1090.40306718365

In [11]:
outside.mean()
# 0.25 1276.75
# 0.2  632.81
# 0.1 209.00

632.8174561004448

In [12]:
(width + outside).mean()
# 0.25 2265.492
# 0.2  1723.220
# 0.1 2036.48

1723.220523284095